# Import relevant libraries for the project

In [1]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

import tqdm

from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay, auc, roc_curve

import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torchvision.datasets import ImageFolder
from torchvision.transforms import v2, ToTensor, Normalize
from torchvision.models import resnet18, ResNet18_Weights
from torchvision.transforms import Grayscale
from PIL import Image

# Preparation

In [2]:
arch = "ResNet18"

In [3]:
# BATCH_SIZE = 8
NUM_WORKERS = 0  # dont change this inside of the jupyter notebook (it will crash)
SEED = 42

MODEL_SAVE_PATH = f'./models/{arch}/best_model.pth'

# Classes of images in test dataset
CLASSES = ['defective', 'ok']
N_CLASSES = len(CLASSES)

In [4]:
# Image size for ResNet
img_size = (224, 224)

In [5]:
# Set seed
torch.manual_seed(SEED)

In [6]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("active device:", device)

active device: cuda


In [7]:
# Define Transforms
transform = v2.Compose([
    ToTensor(),
    v2.Resize(img_size, interpolation=v2.InterpolationMode.NEAREST),
    Grayscale(num_output_channels= 1),
    Normalize([0.5], [0.5]),
])

def target_transform(x):
    return F.one_hot(torch.LongTensor([x]), N_CLASSES)[0].float()

In [8]:
# Define the model
class CustomResNet(nn.Module):
    def __init__(self, num_classes= 2, input_channels= 1):
        super(CustomResNet, self).__init__()
        
        # load the pre-trained ResNet model
        self.base_model = resnet18(weights= ResNet18_Weights.DEFAULT)
        
        # change the input channels of the model
        self.base_model.conv1 = nn.Conv2d(
            input_channels, 64, kernel_size=7, stride=2, padding=3, bias=False
        )
        
        # # Freeze all layers except the last one
        # for param in self.base_model.parameters():
        #     param.requires_grad = False
        
        # change the output layer of the model
        self.base_model.fc = nn.Linear(self.base_model.fc.in_features, num_classes)
    
    def forward(self, x):
        return self.base_model(x)

In [9]:
model = CustomResNet(num_classes= N_CLASSES, input_channels= 1)

model = torch.load(MODEL_SAVE_PATH, weights_only= False, map_location= device)

model = model.to(device)

In [10]:
def predict_single_image(image_path):
    # Load and transform the image
    image = Image.open(image_path)
    input = transform(image).unsqueeze(0).to(device)  # Add batch dimension and move to the device

    # Perform prediction
    model.eval()
    with torch.no_grad():
        output = model(input)
        probabilities = torch.softmax(output.squeeze(), dim=0)  # Convert logits to probabilities
        predicted_class = torch.argmax(probabilities).item()  # Get the predicted class index

    # Print the prediction result and probabilities
    print(f"Predicted Class: {CLASSES[predicted_class]}")
    for i, prob in enumerate(probabilities):
        print(f"Class: {CLASSES[i]}, Probability: {prob.item():.4f}")

In [11]:
image_path = './demo/demo_1.jpeg'
predict_single_image(image_path)

Predicted Class: ok
Class: defective, Probability: 0.0000
Class: ok, Probability: 1.0000


In [12]:
image_path = './demo/demo_2.jpeg'
predict_single_image(image_path)

Predicted Class: defective
Class: defective, Probability: 1.0000
Class: ok, Probability: 0.0000
